# Math AI Playground

- Evaluate numeric-only math expression step by step.
- Expression length cannot be more than 20 characters.
- Support `*` and `+` only, `-` and `()` support will be attempted later. Intended for the neural network to learn the rules of mathematics.
- Support integers only.

Character encoding:

| Character | Token    |
|-----------|----------|
| 0         | 0        |
| 1         | 1        |
| 2         | 2        |
| 3         | 3        |
| 4         | 4        |
| 5         | 5        |
| 6         | 6        |
| 7         | 7        |
| 8         | 8        |
| 9         | 9        |
| *         | 10       |
| +         | 11       |
| -         | 12       |
|  (space)  | 13       |

In [1]:
!nvidia-smi

Fri Oct 21 20:29:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   52C    P8    22W / 170W |    525MiB / 12288MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install tensorflow-text==2.9.0

In [3]:
# Global imports

from typing import List, Tuple

import random
import re

import pandas as pd

import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow import keras
from tensorflow.keras import layers, losses
import numpy as np
import datetime
import os
from tensorflow.keras import mixed_precision

RANDOM_SEED = 1

randomer = random.Random(RANDOM_SEED)

2022-10-21 20:29:16.812969: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
tf.config.list_physical_devices('GPU')

2022-10-21 20:29:17.456273: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-21 20:29:17.456840: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-21 20:29:17.497107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 20:29:17.497207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2022-10-21 20:29:17.497222: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-10-21 20:29:17.498508: I tensorflow/stream_executor/platform/d

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

so returning NUMA node zero
2022-10-21 20:29:17.501387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 20:29:17.501438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1862] Adding visible gpu devices: 0


In [5]:
CHAR_TOKEN_MAP = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '*': 10,
    '+': 11,
    '-': 12,
    ' ': 13
}

TOKEN_CHAR_MAP = {}

for k, v in CHAR_TOKEN_MAP.items():
    TOKEN_CHAR_MAP[v] = k

TOKEN_CHAR_MAP

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '*',
 11: '+',
 12: '-',
 13: ' '}

In [6]:
CHAR_VOCAB = list(CHAR_TOKEN_MAP)
CHAR_VOCAB

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '+', '-', ' ']

## Data generation

In [7]:
DIGIT_CHARS = list('0123456789')
OP_CHARS = list('*+')
EXPRESSION_LENGTH = 20
TOKEN_SPACE_SIZE = 16


def generate_initial_expression(rand: random.Random) -> str:
    initial_expression = [rand.choice(DIGIT_CHARS) if i % 3 < 2 else rand.choice(OP_CHARS) for i in range(14)]
    if initial_expression[0] == '0':
        initial_expression = initial_expression[1:]
    initial_expression = re.sub("([\*\+\-])0", lambda mo: mo.group(1), ''.join(initial_expression))

    return initial_expression


# Evaluate one step of the expression, return expression, False if no further
# steps can be made anymore.
def progress_expression_step(expression: str) -> Tuple[str, bool]:
    op = ''
    op_idx = -1

    if '*' in expression:
        # Multiplication takes precendence
        op = '*'
        op_idx = expression.find('*')
    else:
        m = re.search('\+', expression)
        if m is not None:
            op = expression[m.start()]
            op_idx = m.start()

    if op == ' ' or op_idx == -1:
        return expression, False

    start_idx = op_idx - 1
    end_idx = op_idx + 1

    while start_idx - 1 >= 0 and expression[start_idx - 1] not in OP_CHARS:
        start_idx -= 1

    while end_idx + 1 < len(expression) and expression[end_idx + 1] not in OP_CHARS:
        end_idx += 1

    num1 = int(expression[start_idx:op_idx])
    num2 = int(expression[op_idx + 1:end_idx + 1])

    calc_result = 0

    if op == '*':
        calc_result = num1 * num2
    elif op == '+':
        calc_result = num1 + num2
    elif op == '-':
        calc_result = num1 - num2

    before_result = expression[:start_idx]
    after_result = expression[end_idx + 1:]

    if len(before_result) > 0 and calc_result < 0:
        if before_result[-1] == '-':
            before_result[-1] = '+'
        elif before_result[-1] == '+':
            before_result[-1] = '-'

    calc_result = abs(calc_result)

    return before_result + str(calc_result) + after_result, True


def generate_expression_with_steps(rand: random.Random) -> List[str]:
    initial_expression = generate_initial_expression(rand)
    ret = [initial_expression]
    while True:
        exp, has_further = progress_expression_step(ret[-1])
        if not has_further:
            break
        ret.append(exp)
    if int(eval(initial_expression)) != int(ret[-1]):
        raise ValueError("internal logic error, value evaluation is incorrect")
    return [l.ljust(20) for l in ret]


generate_expression_with_steps(randomer)

['29*41+77+31+6       ',
 '1189+77+31+6        ',
 '1266+31+6           ',
 '1297+6              ',
 '1303                ']

In [8]:

!mkdir -p data /

def regenerate_all_inputs():
    rnd = random.Random(1)

    from_expression_train_file = open('data/from_expression_train.txt', "w")
    to_expression_train_file = open('data/to_expression_train.txt', "w")

    for i in range(10 ** 6):
        steps = generate_expression_with_steps(rnd)
        for j in range(len(steps) - 1):
            from_expression_train_file.write(steps[j] + "\n")
            to_expression_train_file.write(steps[j + 1] + "\n")

    from_expression_train_file.close()
    to_expression_train_file.close()

    from_expression_test_file = open('data/from_expression_test.txt', "w")
    to_expression_test_file = open('data/to_expression_test.txt', "w")

    for i in range(2 * 10 ** 5):
        steps = generate_expression_with_steps(rnd)
        for j in range(len(steps) - 1):
            from_expression_test_file.write(steps[j] + "\n")
            to_expression_test_file.write(steps[j + 1] + "\n")

    from_expression_test_file.close()
    to_expression_test_file.close()

    from_expression_validation_file = open('data/from_expression_validation.txt', "w")
    to_expression_validation_file = open('data/to_expression_validation.txt', "w")

    for i in range(2 * 10 ** 5):
        steps = generate_expression_with_steps(rnd)
        for j in range(len(steps) - 1):
            from_expression_validation_file.write(steps[j] + "\n")
            to_expression_validation_file.write(steps[j + 1] + "\n")

    from_expression_validation_file.close()
    to_expression_validation_file.close()

# regenerate_all_inputs()

In [9]:
% load_ext tensorboard

In [10]:
# Good reference https://www.tensorflow.org/tutorials/load_data/text

PADDED_VOCAB_SIZE = len(CHAR_VOCAB) + 2

keys = [ord(c) for c in CHAR_VOCAB]
values = range(2, len(CHAR_VOCAB) + 2)

init = tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

2022-10-21 20:29:17.762360: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 20:29:17.762871: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-21 20:29:17.762985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 20:29:17.763079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2022-10-21 20

In [11]:
from_expression_train_text_dataset = tf.data.TextLineDataset('data/from_expression_train.txt')
to_expression_train_text_dataset = tf.data.TextLineDataset('data/to_expression_train.txt')
from_expression_validation_text_dataset = tf.data.TextLineDataset('data/from_expression_validation.txt')
to_expression_validation_text_dataset = tf.data.TextLineDataset('data/to_expression_validation.txt')
from_expression_test_text_dataset = tf.data.TextLineDataset('data/from_expression_test.txt')
to_expression_test_text_dataset = tf.data.TextLineDataset('data/to_expression_test.txt')

In [12]:
tokenizer = tf_text.UnicodeCharTokenizer()


def tokenize(text):
    return tokenizer.tokenize(text)


tokenized_example_ds = from_expression_train_text_dataset.map(tokenize)

for text_batch in tokenized_example_ds.take(1):
    print("Tokens: ", text_batch[0])

2022-10-21 20:29:18.209108: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-10-21 20:29:18.227658: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2496000000 Hz


Tokens:  tf.Tensor(50, shape=(), dtype=int32)


In [13]:
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://stackoverflow.com/questions/41399481/how-do-you-decode-one-hot-labels-in-tensorflow

def preprocess_text(text):
    standardized = tf_text.case_fold_utf8(text)
    tokenized = tokenizer.tokenize(standardized)
    tokenized = tf.cast(tokenized, tf.int64)
    vectorized = vocab_table.lookup(tokenized)
    encoded = tf.one_hot(vectorized, PADDED_VOCAB_SIZE)
    padded = tf.reshape(encoded, (20, 16))
    return padded


example_text = next(iter(from_expression_train_text_dataset))
example_text.numpy()

preprocess_text(example_text)


<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
 

In [14]:
from_expression_train_encoded = from_expression_train_text_dataset.map(preprocess_text)
to_expression_train_encoded = to_expression_train_text_dataset.map(preprocess_text)
from_expression_test_encoded = from_expression_test_text_dataset.map(preprocess_text)
to_expression_test_encoded = to_expression_test_text_dataset.map(preprocess_text)
from_expression_validation_encoded = from_expression_validation_text_dataset.map(preprocess_text)
to_expression_validation_encoded = to_expression_validation_text_dataset.map(preprocess_text)

In [15]:
next(iter(from_expression_train_encoded))

<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
 

In [16]:
train_data = tf.data.Dataset.zip((from_expression_train_encoded, to_expression_train_encoded))
train_data

<ZipDataset shapes: ((20, 16), (20, 16)), types: (tf.float32, tf.float32)>

In [17]:
validation_data = tf.data.Dataset.zip((from_expression_validation_encoded, to_expression_validation_encoded))
validation_data

<ZipDataset shapes: ((20, 16), (20, 16)), types: (tf.float32, tf.float32)>

In [18]:
test_data = tf.data.Dataset.zip((from_expression_test_encoded, to_expression_test_encoded))
test_data

<ZipDataset shapes: ((20, 16), (20, 16)), types: (tf.float32, tf.float32)>

In [45]:
BATCH_SIZE = 32
train_data_batched = train_data.padded_batch(BATCH_SIZE)
validation_data_batched = validation_data.padded_batch(BATCH_SIZE)
test_data_batched = test_data.padded_batch(BATCH_SIZE)

In [20]:
next(iter(from_expression_train_encoded))

<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
 

In [21]:
def create_model():
    return tf.keras.Sequential([
        layers.Input(shape=(20, 16)),
        layers.Flatten(),
        layers.Dense(20 * 16 * 10),
        layers.Dense(20 * 16 * 10),
        layers.Dense(20 * 16, activation='relu'),
        layers.Reshape((20, 16)),
        layers.Softmax(axis=2)
    ])

In [22]:
softmax_layer = layers.Softmax(axis=1)
test_input = np.zeros((20, 16))
test_input[19, 1] = 3
test_input[19, 2] = 2
softmax_layer(test_input)[19]

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.02411114, 0.4842853 , 0.17815863, 0.02411114, 0.02411114,
       0.02411114, 0.02411114, 0.02411114, 0.02411114, 0.02411114,
       0.02411114, 0.02411114, 0.02411114, 0.02411114, 0.02411114,
       0.02411114], dtype=float32)>

In [23]:
model = create_model()
model

In [67]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

model.compile(
    optimizer='adam',
    loss=losses.MeanSquaredError(),

)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

2022-10-22 10:43:49.953801: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-10-22 10:43:49.953821: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-10-22 10:43:50.014842: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-10-22 10:43:50.014939: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed


In [25]:
# https://stackoverflow.com/questions/44607176/tensorflow-loss-function-which-takes-one-hot-as-argument
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy
# Would CategoricalCrossentrophy with axis = 1 work?

class SoftmaxCrossEntrophyError2D(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        pass

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 320)               0         
_________________________________________________________________
dense (Dense)                (None, 3200)              1027200   
_________________________________________________________________
dense_1 (Dense)              (None, 3200)              10243200  
_________________________________________________________________
dense_2 (Dense)              (None, 320)               1024320   
_________________________________________________________________
reshape (Reshape)            (None, 20, 16)            0         
_________________________________________________________________
softmax_1 (Softmax)          (None, 20, 16)            0         
Total params: 12,294,720
Trainable params: 12,294,720
Non-trainable params: 0
____________________________________________

In [27]:
# from google.colab import drive
# drive.mount('/content/drive')

In [70]:
checkpoint_path = "training_9/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [71]:
optimizer_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 2
)

history = model.fit(
    train_data_batched,
    validation_data=validation_data_batched,
    epochs=100,
    callbacks=[tensorboard_callback, cp_callback, optimizer_scheduler_callback],
)

Epoch 1/100
      3/Unknown - 1s 135ms/step - loss: 0.0152

2022-10-22 10:44:25.731969: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-10-22 10:44:25.731988: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-10-22 10:44:25.842824: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-10-22 10:44:25.843027: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed
2022-10-22 10:44:25.972363: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 134 callback api events and 126 activity events. 
2022-10-22 10:44:25.974809: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-10-22 10:44:25.978636: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/fit/20221022-104349/train/plugins/profile/2022_10_22_10_44_25
2022-10-22 10:44:25.982570: I tensorflow/core/profiler/rpc/client/save_profile.cc:143

125000/125000 [==============================] - 640s 5ms/step - loss: 0.0160 - val_loss: 0.0160

Epoch 00001: saving model to training_9/cp-0001.ckpt
Epoch 2/100
125000/125000 [==============================] - 644s 5ms/step - loss: 0.0160 - val_loss: 0.0160

Epoch 00002: saving model to training_9/cp-0002.ckpt
Epoch 3/100
125000/125000 [==============================] - 652s 5ms/step - loss: 0.0160 - val_loss: 0.0160

Epoch 00003: saving model to training_9/cp-0003.ckpt
Epoch 4/100
125000/125000 [==============================] - 655s 5ms/step - loss: 0.0160 - val_loss: 0.0160

Epoch 00004: saving model to training_9/cp-0004.ckpt
Epoch 5/100
125000/125000 [==============================] - 646s 5ms/step - loss: 0.0160 - val_loss: 0.0160

Epoch 00005: saving model to training_9/cp-0005.ckpt
Epoch 6/100
125000/125000 [==============================] - 659s 5ms/step - loss: 0.0160 - val_loss: 0.0160

Epoch 00006: saving model to training_9/cp-0006.ckpt
Epoch 7/100
125000/125000 [=======

KeyboardInterrupt: 

In [61]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_8/cp-0070.ckpt'

In [ ]:
model = create_model()

In [68]:
model.load_weights(latest)

In [63]:
model.predict(validation_data_batched.take(1))

array([[[6.250e-02, 6.250e-02, 6.250e-02, ..., 6.250e-02, 6.250e-02,
         6.250e-02],
        [6.250e-02, 6.250e-02, 6.250e-02, ..., 6.250e-02, 6.250e-02,
         6.250e-02],
        [0.000e+00, 4.208e-03, 2.158e-01, ..., 0.000e+00, 3.714e-03,
         1.205e-03],
        ...,
        [5.960e-08, 5.960e-08, 1.788e-07, ..., 5.960e-08, 5.960e-08,
         1.000e+00],
        [5.960e-08, 5.960e-08, 5.960e-08, ..., 5.960e-08, 5.960e-08,
         1.000e+00],
        [1.192e-07, 1.788e-07, 5.960e-08, ..., 5.960e-08, 5.960e-08,
         1.000e+00]],

       [[6.250e-02, 6.250e-02, 6.250e-02, ..., 6.250e-02, 6.250e-02,
         6.250e-02],
        [1.549e-01, 1.549e-01, 1.503e-04, ..., 0.000e+00, 0.000e+00,
         1.549e-01],
        [6.250e-02, 6.250e-02, 6.250e-02, ..., 6.250e-02, 6.250e-02,
         6.250e-02],
        ...,
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         1.000e+00],
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
   

In [72]:
PREDICT_KEYS = [ord('P'), ord('X')] + keys


def reverse_preprocess_text(prediction):
    vectorized = tf.argmax(prediction, axis=1)
    tokenized = tf.map_fn(lambda e: PREDICT_KEYS[e], vectorized)
    tokenized = tf.cast(tokenized, tf.int32)
    return tokenizer.detokenize(tokenized)

In [73]:
# prediction = model.predict(train_data_batched.take(1))
# [reverse_preprocess_text(p).numpy() for p in prediction]

def visualize_prediction(batched_data, t):
    for batch in batched_data.take(t):
        (inputs, targets) = batch
        predicts = model.predict_on_batch(inputs)
        lines = zip(inputs, targets, predicts)
        for line in lines:
            print(reverse_preprocess_text(line[0]).numpy(), reverse_preprocess_text(line[1]).numpy(),
                  reverse_preprocess_text(line[2]).numpy())

In [74]:
visualize_prediction(train_data_batched, 1)

b'29*41+77+31+6       ' b'1189+77+31+6        ' b'PP8P+77+3++6        '
b'1189+77+31+6        ' b'1266+31+6           ' b'1PPP+31+P           '
b'1266+31+6           ' b'1297+6              ' b'12P6+6              '
b'1297+6              ' b'1303                ' b'1297                '
b'69*74*91+0*80       ' b'5106*91+0*80        ' b'PP6P*91+0*80        '
b'5106*91+0*80        ' b'464646+0*80         ' b'P5PP00+8*80         '
b'464646+0*80         ' b'464646+0            ' b'4646P0+000          '
b'464646+0            ' b'464646              ' b'4646PP              '
b'63+8*77*53*74       ' b'63+616*53*74        ' b'63+PPP*33*74        '
b'63+616*53*74        ' b'63+32648*74         ' b'63+2PPPP*74         '
b'63+32648*74         ' b'63+2415952          ' b'63+1PPPPP0          '
b'63+2415952          ' b'2416015             ' b'2PPPPPP             '
b'6*24*58+83+49       ' b'144*58+83+49        ' b'PPP*PP+83+49        '
b'144*58+83+49        ' b'8352+83+49          ' b'PPPP0P83+49   

In [75]:
visualize_prediction(validation_data_batched, 1)

b'63*50*56*83+65      ' b'3150*56*83+65       ' b'PP00*56*83+65       '
b'3150*56*83+65       ' b'176400*83+65        ' b'P1PP00*83+65        '
b'176400*83+65        ' b'14641200+65         ' b'P5610000+6          '
b'14641200+65         ' b'14641265            ' b'1464PPP             '
b'30+26+60+15+86      ' b'56+60+15+86         ' b'1P+P6+P5+PP P       '
b'56+60+15+86         ' b'116+15+86           ' b'1PP+15+8            '
b'116+15+86           ' b'131+86              ' b'1PP+86              '
b'131+86              ' b'217                 ' b'2PPP                '
b'16+18+78*33+91      ' b'16+18+2574+91       ' b'16+18+PPPP99        '
b'16+18+2574+91       ' b'34+2574+91          ' b'PP+P87PP+P          '
b'34+2574+91          ' b'2608+91             ' b'2PPP+91             '
b'2608+91             ' b'2699                ' b'26PP                '
b'7+18*61*51+15       ' b'7+1098*51+15        ' b'7+PPPP*11+15        '
b'7+1098*51+15        ' b'7+55998+15          ' b'7+1PPPPP+15   

In [76]:
visualize_prediction(test_data_batched, 1)

b'50+23+72+85*11      ' b'50+23+72+935        ' b'50+2P+PP+PPPP       '
b'50+23+72+935        ' b'73+72+935           ' b'PP+P7+P3P           '
b'73+72+935           ' b'145+935             ' b'1PP+9PP             '
b'145+935             ' b'1080                ' b'PPPP                '
b'41*53+11*98*11      ' b'2173+11*98*11       ' b'PP5P+11*98*11       '
b'2173+11*98*11       ' b'2173+1078*11        ' b'2173+PPP**1         '
b'2173+1078*11        ' b'2173+11858          ' b'2173+PP0PP          '
b'2173+11858          ' b'14031               ' b'PPPPP               '
b'84+19+60+94+84      ' b'103+60+94+84        ' b'1PP+60+9+PP P       '
b'103+60+94+84        ' b'163+94+84           ' b'1P3+94+8P           '
b'163+94+84           ' b'257+84              ' b'2PP+84              '
b'257+84              ' b'341                 ' b'+PPP                '
b'43+85+23+38*71      ' b'43+85+23+2698       ' b'43+85+PP+PPPP       '
b'43+85+23+2698       ' b'128+23+2698         ' b'1PP+2P+2PPP   